In [14]:
import requests
from bs4 import BeautifulSoup


class LLMBrowser:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
    def __call__(self, request: str):
        q = "+".join(request.split())
        r = requests.get(f"https://www.google.com/search?q={q}")
        soup = BeautifulSoup(r.text, features="html.parser")

        soup = self.del_script_and_style(soup)
        text = self.extract_text(soup)
        return text

    def del_script_and_style(self, soup):
        for script in soup(["script", "style"]):
            script.extract()    # rip it out
        
        return soup
        
    def extract_text(self, soup):
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        
        return text

In [16]:
browser = LLMBrowser(None, None)
browser("Гостиница Мрия это")

'Гостиница Мрия это - Поиск в GoogleGoogle×Нажмите здесь, если переадресация не будет выполнена в течение нескольких секунд.\nВсеКартинкиКартыВидео Новости Покупки Книги Инструменты поиска\nНа всех языкахНа всех языкахТолько на русскийЗа всё времяЗа всё время За час За 24 часа За неделю За месяц За годВсе результатыВсе результатыТочное соответствиеО курорте Mriya Resort & SPAmriyaresort.com › aboutMriya Resort & SPA — ведущий мировой курорт премиум-класса, расположенный на Южном берегу Крыма между живописным горным массивом и бескрайним морским\xa0...МРИЯ РЕЗОРТ энд СПА 4,7\n(4\xa0325)\nОтельМаршрутРоскошный пляжный спа-отель с крытым и открытым бассейнами, а также несколькими ресторанами и барами.Адрес: ул. Генерала Острякова, 9, ОползневоеТелефон: 8 (800) 550-52-71Вопросы по темеКому принадлежит гостиница Мрия?Что было на месте отеля Мрия?Кто держит мрию в Крыму?Что входит в полный пансион Мрия?Официальный сайт курорта Mriya Resort & SPA 5* в Крымуmriyaresort.comОфициальный сайт Mriy

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [ ]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])
